In [ ]:
using Pkg,Statistics,Random,LinearAlgebra,Printf,GZip,Knet,Images,Distributions,Plots #dy: do you need all this?

In [ ]:
L=31; # number of training data points

In [ ]:
Range=2.0; # range of the x values

In [ ]:
NoiseRatio=0.3; # max noise/value

In [ ]:
function gen_noisy_gaussian(len;range=1.0,noiseratio=0.1)
    x = sort(randn(len)*range) #dy:randn does the same thing. # sort(rand(Normal(0,1),len)*range)
    y = exp.(-x.^2).*(-noiseratio*2*(rand(len).-0.5).+1) # fractional noise
    return (x,y)
end

In [ ]:
(x_train,y_train) = gen_noisy_gaussian(L,range=Range,noiseratio=NoiseRatio);

In [ ]:
plot(x_train,[y_train,exp.(-x_train.^2)])

In [ ]:
x_train = permutedims(x_train)
y_train = permutedims(y_train)

In [ ]:
Layersize = 3; # number of neurons in a layer

In [ ]:
# output = 1
# input = 1
# hidden = 3
# batchsize = 31
#relu.(w[hidden,input] * x[input,batchsize] .+ b[hidden,1])

In [ ]:
w = [0.1*rand(Layersize,1),0.1*rand(Layersize,1),0.1*rand(1,Layersize),0.1*rand(1,1)]

In [ ]:
guesses = sum(w[3] * tanh.(w[1]*x.+w[2]) .+ w[4],dims=1) # w[1]=w, w[2]=w0, w[3]=w', w[4]=w0'

In [ ]:
#w[1]: (1,3)
#x: (31,1)
#dy: did you really mean to use .* here?  neural net layers use matmul. also biases are lower dim.
function loss(w,x,y)
    guesses = sum(w[3] * tanh.(w[1]*x.+w[2]) .+ w[4],dims=1) # w[1]=w, w[2]=w0, w[3]=w', w[4]=w0'
    return mean(abs2,y-guesses)
end

In [ ]:
loss(w,x,y)

In [ ]:
lossgradient = grad(loss)

In [ ]:
dw = lossgradient(w,x_train,y_train)

In [ ]:
function mytrain!(w;lr=0.1)
    dw = lossgradient(w,x_train,y_train);
    for i = 1:length(w)
        for j = 1:length(w[i])
            w[i][j] -= lr*dw[i][j];
        end
    end
    return w
end

In [ ]:
weights = [ deepcopy(mytrain!(w,lr=0.1)) for epoch=1:10 ]  # copy only copies the top layer, does not descend.

In [ ]:
# w[hidden, input] * x[input, batchsize] => z[hidden,batchsize] .+ b[hidden,1] => y[hidden,batchsize]

x1 = input
#x1 = dropout(x1,0.5)
x2 = tanh.(w1 * x1 .+ b1)
x2 = dropout(x2,0.5)
x3 = tanh.(w2 * x2 .+ b2)
#x3 = dropout(x3,0.5)

In [ ]:
@doc dropout